In [15]:
#from helper_funcs import bayes_step
import fastdyn_fic_dmf as dmf
from mango import Tuner, scheduler
from scipy.io import loadmat
import numpy as np

N_JOBS = 4


@scheduler.parallel(n_jobs=N_JOBS)
def bayes_step(G, DECAY, LR):
    OBJ_RATE = 3.44
    sampling_freq = 10000
    nb_steps = 100000
    C = loadmat('../SC_and_5ht2a_receptors.mat')['sc90']
    C = 0.2*C/np.max(C)
    triu_idx = np.triu_indices(C.shape[1],1)
    brunout = 5
    params = dmf.default_params(C=C)
    params['N'] = C.shape[0]
    #print(f"Running - G:{G} / DECAY:{DECAY} / OBJ_RATE:{OBJ_RATE} / LR:{LR} \n")
    with_decay = DECAY>0
    params['lrj'] = LR
    params['G'] = G
    params['taoj'] = DECAY if with_decay else 10 # If 0 it means no plasticity at all. We put some value so it does not crash
    params['obj_rate'] = OBJ_RATE
    #params['taoj'] = 210000
    params['J'] = 0.75*params['G']*params['C'].sum(axis=0).squeeze() + 1
    rates, rates_inh, _, fic_t = dmf.run(params, nb_steps,
                                            return_rate=True, return_bold=False, return_fic=True, 
                                            with_decay=with_decay, with_plasticity=True)        
    rates = rates[:, np.ceil(brunout * 1000).astype(int):]
  
    homeostatic_fittness =  np.abs(OBJ_RATE - np.mean(rates))
    return homeostatic_fittness

In [32]:
# Bounded region of parameter space
lr_range = np.concatenate([np.arange(0.1, 1.1, 0.1), np.array(range(2, 401, 2))])
pbounds = {'G': np.arange(1,6.5,0.5), 'DECAY': range(0,200,100001), 'LR':lr_range}
conf_dict = dict(num_iteration=3, initial_random=10)

In [33]:
tuner = Tuner(pbounds, bayes_step, conf_dict)
results = tuner.maximize()
print('best parameters:', results['best_params'])
print('best accuracy:', results['best_objective'])

  0%|          | 0/40 [00:00<?, ?it/s]


KeyboardInterrupt: 